Dati

In [ ]:
import json

with open('/home/gbarbaro/UniQA/lora_ft_reduced_chat/it_test.json', 'r') as file:
    data = json.load(file)

In [ ]:
def trova_indice_domanda(domanda):
    for i, item in enumerate(data):
        if item['question'] == domanda:
            return i
    return -1

domanda_input = ""
indice = trova_indice_domanda(domanda_input)

if indice != -1:
    print(f"L'indice della domanda è: {indice}")
else:
    print("Domanda non trovata nel file JSON.")


In [ ]:
index = [0, 1, 1156, 1158, 1162, 1168, 1170, 1184, 1238, 1241]

In [ ]:
index = list(range(1249))

In [ ]:
from ragas.metrics import Faithfulness, AnswerCorrectness
from ragas import evaluate
import nest_asyncio
import os
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
    
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = "token"
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

In [ ]:
import json
from datasets import Dataset 

# Carica i dataset di valutazione
with open('/home/gbarbaro/Test/Inferenza/output_metric_total/output_metric_total_ANITA.json', 'r') as file:
    data = json.load(file)

j=0
output = [data[j]['output_pairs']['output'][i] for i in index] 
reference = [data[j]['output_pairs']['reference'][i] for i in index]

with open('/home/gbarbaro/UniQA/lora_ft_reduced_chat/it_test.json', 'r') as file:
    data_input = json.load(file)

user_input = [data_input[i]["question"] for i in index]

retrieved_contexts = []

for i in index:
    if "Documents" in data_input[i]["input"]: 
        indice = data_input[i]["input"].find("Documents") 
        retrieved_contexts.append(data_input[i]["input"][indice + len("Documents:\n"):].strip().split('\n')) 
    else: retrieved_contexts.append([])

# Converti output_pairs in un formato compatibile con Hugging Face Dataset 
dataset_dict = {"user_input": user_input, "reference": reference, "response": output, "retrieved_contexts": retrieved_contexts} 
hf_dataset = Dataset.from_dict(dataset_dict)

In [ ]:
import evaluate

def evaluate_metrics(references, predictions):
    bleu = evaluate.load("bleu")
    bleu_score = bleu.compute(predictions=predictions, references=references)

    rouge = evaluate.load('rouge')
    rouge_score = rouge.compute(predictions=predictions,references=references)
    
    bertscore = evaluate.load("bertscore")
    bert_score = bertscore.compute(predictions=predictions, references=references, lang="it", device="cuda")
    
    meteor = evaluate.load('meteor')
    meteor_score = meteor.compute(predictions=predictions, references=references)
    
    return bleu_score, rouge_score, bert_score, meteor_score

def mean(list):
    mean = sum(list)/len(list)
    
    return mean
    
output_pairs = {'output': hf_dataset["response"], 'reference': hf_dataset["reference"]}

bleu_score, rouge_score, bert_score, meteor_score = evaluate_metrics(output_pairs['reference'], output_pairs['output'])
    
print(f"BLEU score: {round(bleu_score['bleu'], 2)}")
print(f"ROUGE score: {rouge_score}")
print(f"BERT_PRECISION score: {round(mean(bert_score['precision']), 2)}")
print(f"BERT_RECALL score: {round(mean(bert_score['recall']), 2)}")
print(f"BERT_F1 score: {round(mean(bert_score['f1']), 2)}")
print(f"METEOR score: {round(meteor_score['meteor'], 2)}")

output_metric = {'output_pairs': output_pairs,
                'bleu_score': bleu_score,
                'rouge_score': rouge_score,
                'bert_score': bert_score,
                'meteor_score': meteor_score,
                'perplexity_score': 0,
                'perplexity_z_score': 0
                }
    
try:
    with open('output_metric.json', 'r') as file:
        output_metric_json = json.load(file)
except FileNotFoundError:
    output_metric_json = []
except json.JSONDecodeError:
    output_metric_json = []

output_metric_json.append(output_metric)
    
with open('output_metric_10.json', 'w') as file:
    json.dump(output_metric_json, file, indent=4)

In [ ]:
metrics = [
    AnswerCorrectness(llm = evaluator_llm),
    Faithfulness(llm = evaluator_llm),
]

results = evaluate(dataset=hf_dataset, metrics=metrics, raise_exceptions=False)

print(results)

Voto Soggettivo

In [ ]:
import json

with open('/home/gbarbaro/Test/Metriche-LLM-Z-GPT/output_total.json', 'r') as file:
    data_r = json.load(file)

In [ ]:
import json

with open('/home/gbarbaro/Test/Inferenza_Fine_Tuning/Risultati_FT/output_metric_LLAMA_3.3_FT.json', 'r') as file:
    data = json.load(file)

In [ ]:
index = [0, 1, 1156, 1158, 1162, 1168, 1170, 1184, 1238, 1241]

In [ ]:
print(data[0]['output_pairs']['output'][1241])

In [ ]:
print(data_r['reference'][1241])